In [47]:
# !pip install PyPDF2
# !pip install nltk
import nltk
import re
import json
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Paragraph, SimpleDocTemplate
from reportlab.lib.styles import getSampleStyleSheet
import openai
import requests

In [264]:
def query_chatgpt(prompt, model="gpt-3.5-turbo"):
    
    URL = "https://api.openai.com/v1/chat/completions" 
    # openai.api_key = 

    payload = {
               "model": "gpt-3.5-turbo",
               "temperature" : 0,
               "messages" :[
                           {"role": "system", "content":  "You are an expert linguist and author. When given a sentence to summarise, please return \
                                                           the words in the sentence that best capture the meaning of it. The words that you choose \
                                                           to return must be done so in the same order as they appeared in the sentence. Additionally, \
                                                           please return the numerical position of the chosen words within the sentence. The output should \
                                                           be amaximum of 4 words and in list format. \
                                                           For an example, if given the following sentence \
                                                           'An AI learned to win chess by making moves human grand masters had never conceived.' \
                                                           The output should be ['AI', 'learned', 'chess'] \
                                                           Another example, if given the following sentence \
                                                           'In indian philosophy, enlightenment is seen to be ineffable.' \
                                                           The output should be ['indian', 'philosophy', 'enlightenment', 'ineffable'] \
                                                           Another example, if given the following sentence \
                                                           'In the 20th century, we learnt that war is never the answer, but it still remains a possibility.' \
                                                           The output should be ['war', 'never', 'answer'] \
                                                           Another example, if given the following sentence \
                                                           'One of the most important things to build for a high performing data team, is an experimentation platform.' \
                                                           The output should be ['important', 'data', 'team', 'experimentation'] \
                                                           Another example, if given the following sentence \
                                                           'The pay back period is the most important metrics for any business, regardless of industry or size.' \
                                                           The output should be ['pay', 'back', 'important', 'metrics']"},
                            {"role": "user", "content": prompt},
                            ] 
              }
    
    headers = {
              "Content-Type": "application/json",
              "Authorization": f"Bearer {openai.api_key}"
              }

    response = requests.post(URL, headers=headers, json=payload)
    response = response.json()


    return response['choices'][0]['message']['content']

In [265]:
prompt = "I like to leave examples so you can see how it works."

response = query_chatgpt(prompt)

print(response)

['leave', 'examples', 'works']


In [266]:
                                                                    # Doing it manually 
# Open the PDF file
with open('/Users/suleman.farooqi/test.pdf', 'rb') as file:
    # Create a PDF reader
    reader = PyPDF2.PdfReader(file)
    # Get the content of the first page
    page = reader.pages[0]
    pdf_content = page.extract_text()



# Bold these words (I've chosen them manually)
words = pdf_content.split()

for i in [1, 2, 5, 16, 17, 20, 31, 32, 41, 58, 63]:
    words[i] = '<b>' + words[i] + '</b>'


# Combine the words back into a string
modified_text = ' '.join(words)

# Create a new PDF with the modified text
styles = getSampleStyleSheet()
styleN = styles['BodyText']

story = []

modified_text_paragraph = Paragraph(modified_text, styleN)
story.append(modified_text_paragraph)

pdf = SimpleDocTemplate('/Users/suleman.farooqi/test_output_manual.pdf', pagesize=letter)
pdf.build(story)


In [267]:
                                                                    # Doing it with GPT 
# Open the PDF file
with open('/Users/suleman.farooqi/test.pdf', 'rb') as file:
    # Create a PDF reader
    reader = PyPDF2.PdfReader(file)
    # Get the content of the first page
    page = reader.pages[0]
    pdf_content = page.extract_text()


# Initialise the list 
results = []

# Query chatGPT to find the best words to bold
sentences = nltk.tokenize.sent_tokenize(pdf_content)

for i, sentence in enumerate(sentences):
    
    # Call OpenAI API to identify the words in the given sentence that should be bold     
    bold_words = query_chatgpt(sentence)

    # Create a list of words for the sentence, and highlight the identified words as bold
    words = sentence.split()
    words = ['<b>' + word + '</b>' if word in (bold_words) else word for word in words]
    modified_text = ' '.join(words)

    # Append to results
    styles = getSampleStyleSheet()
    styleN = styles['BodyText']
    modified_text_paragraph = Paragraph(modified_text, styleN)
    results.append(modified_text_paragraph)

pdf = SimpleDocTemplate('/Users/suleman.farooqi/test_output_gpt.pdf', pagesize=letter)
pdf.build(results)

    